<a href="https://colab.research.google.com/github/marialuquea/space_api_colab/blob/main/space_diss_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# API for Space Diss Maria

In [5]:
%%capture
!pip install farm-haystack

In [6]:
%%capture
!pip install pinecone-client

In [7]:
%%capture
!pip install fastapi

In [8]:
%%capture
!pip install colabcode

In [9]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [35]:
import sys
sys.path.append('/content/gdrive/MyDrive/Dissertation')
from qa_text import get_document_store, get_retriever, get_reader, create_pipeline, ask_question
from PineconeDocumentStore_m import *
from types_m import *
from ff_m import _format_filters

In [ ]:
!ls

In [ ]:
!pwd

In [11]:
!cp /content/gdrive/MyDrive/Dissertation/pinecone_document_store.db /content/ 

In [ ]:
!ls

In [12]:
# initialise document store
document_store = PineconeDocumentStore(
        api_key='2622f896-0c2f-45a4-bfff-b4a8577e54d2',
        index='testin4',
        similarity="cosine",
        embedding_dim=768
    )

INFO - haystack.telemetry -  Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by calling disable_telemetry() or by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems on the documentation page. More information at https://haystack.deepset.ai/guides/telemetry


Index statistics: name: testin4, embedding dimensions: 768, record count: 697


In [ ]:
# get retrievers
retriever1, retriever2 = get_retriever(document_store)

In [14]:
%%capture
# get reader
reader = get_reader()

INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find deepset/roberta-base-squad2 locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...
INFO - haystack.modeling.model.language_model -  Loaded deepset/roberta-base-squad2
INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0
INFO - haystack.modeling.infer -  Got ya 2 parallel workers to do inference ...
INFO - haystack.modeling.infer -   0     0  
INFO - haystack.modeling.infer -  /w\   /w\ 
INFO - haystack.modeling.infer -  /'\   / \ 


In [15]:
# combine them into a pipeline
pipeline = create_pipeline(reader, retriever2)

---> Creating extractive QA Pipeline from reader and retriever


In [16]:
pipeline

In [17]:
# Test the pipeline and see what the result looks like
question = "What is space debris?"
prediction = pipeline.run(
        query=question, params={"Retriever": {"top_k": 3}, "Reader": {"top_k": 5}}
    )
prediction

{'answers': [<Answer {'answer': 'a problem', 'type': 'extractive', 'score': 0.23694034665822983, 'context': 'This article considers why space debris is a problem, how space  debris is distributed, what risks space debris poses to satellites, what  internation', 'offsets_in_document': [{'start': 43, 'end': 52}], 'offsets_in_context': [{'start': 43, 'end': 52}], 'document_id': 'af74b43e88ba15d077e3a03249fedf94', 'meta': {'article_title': 'sd_paper_59_m.txt'}}>,
  <Answer {'answer': 'Space Traffic Management', 'type': 'extractive', 'score': 0.00607416033744812, 'context': 'What is Space Traffic Management ', 'offsets_in_document': [{'start': 8, 'end': 32}], 'offsets_in_context': [{'start': 8, 'end': 32}], 'document_id': 'a2249fb45863b927bb51bc7185aae83c', 'meta': {'article_title': 'IAASS_paper_135_m.txt'}}>,
  <Answer {'answer': 'SPACE OBJECT', 'type': 'extractive', 'score': 0.005973541876301169, 'context': 'B. WHAT IS A SPACE OBJECT   ', 'offsets_in_document': [{'start': 13, 'end': 25}],

In [18]:
from fastapi import FastAPI, Response, status
from fastapi.testclient import TestClient
import pickle
import haystack
import time
import json 
import haystack
from numpy import ndarray

In [33]:
app = FastAPI()
client = TestClient(app)

@app.get('/ping')
def index():
    return {'pong'}

@app.get('/initialized', status_code=200)
def index(response: Response):
  response.status_code = status.HTTP_201_CREATED
  return "hola", 200

@app.post("/fakequery/", response_model=QueryResponse)
def create_item(query: QueryResponse):
    print("hello1")
    print(query)
    return query

@app.post("/query/", response_model=QueryResponse, response_model_exclude_none=True)
def query(request: QueryRequest):

    start_time = time.time()

    params = request.params or {}

    # format global, top-level filters (e.g. "params": {"filters": {"name": ["some"]}})
    if "filters" in params.keys():
        params["filters"] = _format_filters(params["filters"])

    # format targeted node filters (e.g. "params": {"Retriever": {"filters": {"value"}}})
    for key in params.keys():
        if "filters" in params[key].keys():
            params[key]["filters"] = _format_filters(params[key]["filters"])

    result = pipeline.run(query=request.query, params=params, debug=request.debug)

    # Ensure answers and documents exist, even if they're empty lists
    if not "documents" in result:
        result["documents"] = []
    if not "answers" in result:
        result["answers"] = []

    # if any of the documents contains an embedding as an ndarray the latter needs to be converted to list of float
    for document in result["documents"]:
        if isinstance(document.embedding, ndarray):
            document.embedding = document.embedding.tolist()

    return result



In [26]:
response = client.get("/ping")
print(response.status_code)
print(response.json())

200
['pong']


In [21]:
response = client.post(
    "/fakequery/",
    # headers={"X-Token": "coneofsilence"},
    json={"query": "how are you?"},
)
print(response.status_code, response.json())

hello1
query='how are you?' answers=[] documents=[] debug=None
200 {'query': 'how are you?', 'answers': [], 'documents': [], '_debug': None}


In [41]:
response = client.post(
    "/query/",
    json={'query': 'What is space debris?', 'params': {'filters': {}, 'Retriever': {'top_k': 3}, 'Reader': {'top_k': 3}}},
)
print(response.json)
# print(response.status_code, response.json())

--> Running a query: query='What is space debris?' params={'filters': {}, 'Retriever': {'top_k': 3}, 'Reader': {'top_k': 3}} debug=False
--> Result from query: {'query': 'What is space debris?', 'no_ans_gap': -10.13113260269165, 'answers': [<Answer {'answer': 'Space Traffic Management', 'type': 'extractive', 'score': 0.00607416033744812, 'context': 'What is Space Traffic Management ', 'offsets_in_document': [{'start': 8, 'end': 32}], 'offsets_in_context': [{'start': 8, 'end': 32}], 'document_id': 'a2249fb45863b927bb51bc7185aae83c', 'meta': {'article_title': 'IAASS_paper_135_m.txt'}}>, <Answer {'answer': 'SPACE OBJECT', 'type': 'extractive', 'score': 0.005973541876301169, 'context': 'B. WHAT IS A SPACE OBJECT   ', 'offsets_in_document': [{'start': 13, 'end': 25}], 'offsets_in_context': [{'start': 13, 'end': 25}], 'document_id': 'c483a71487227da30562ff88bf0dbda3', 'meta': {'article_title': 'IAASS_paper_141_m.txt'}}>, <Answer {'answer': 'Risk', 'type': 'extractive', 'score': 0.00030151756

In [ ]:
response.json()

In [66]:
from haystack.schema import Label
from typing import Any, Dict, List, Union, Optional

@app.post("/feedback/")
def post_feedback(feedback: Union[LabelSerialized, CreateLabelSerialized]):
    if feedback.origin is None:
        feedback.origin = "user-feedback"
    label = Label(**feedback.dict())
    document_store.write_labels([label])

@app.get("/feedback/", response_model=List[LabelSerialized])
def get_feedback():
    labels = document_store.get_all_labels()
    return labels

@app.delete("/feedback/")
def delete_feedback():
    all_labels = document_store.get_all_labels()
    user_label_ids = [label.id for label in all_labels if label.origin == "user-feedback"]
    document_store.delete_labels(ids=user_label_ids)

@app.post("/eval-feedback/")
def get_feedback_metrics(filters: FilterRequest = None):
    if filters:
        filters_content = filters.filters or {}
        filters_content["origin"] = ["user-feedback"]
    else:
        filters_content = {"origin": ["user-feedback"]}
    labels = document_store.get_all_labels(filters=filters_content)
    res: Dict[str, Optional[Union[float, int]]]
    if len(labels) > 0:
        answer_feedback = [1 if l.is_correct_answer else 0 for l in labels]
        doc_feedback = [1 if l.is_correct_document else 0 for l in labels]
        answer_accuracy = sum(answer_feedback) / len(answer_feedback)
        doc_accuracy = sum(doc_feedback) / len(doc_feedback)
        res = {"answer_accuracy": answer_accuracy, "document_accuracy": doc_accuracy, "n_feedback": len(labels)}
    else:
        res = {"answer_accuracy": None, "document_accuracy": None, "n_feedback": 0}
    return res

@app.get("/export-feedback/")
def export_feedback(context_size: int = 100_000, full_document_context: bool = True, only_positive_labels: bool = False):
    """
    This endpoint returns JSON output in the SQuAD format for question/answer pairs
    that were marked as "relevant" by user feedback through the `POST /feedback` endpoint.
    The context_size param can be used to limit response size for large documents.
    """
    if only_positive_labels:
        labels = document_store.get_all_labels(filters={"is_correct_answer": [True], "origin": ["user-feedback"]})
    else:
        labels = document_store.get_all_labels(filters={"origin": ["user-feedback"]})
        # Filter out the labels where the passage is correct but answer is wrong (in SQuAD this matches
        # neither a "positive example" nor a negative "is_impossible" one)
        labels = [l for l in labels if not (l.is_correct_document is True and l.is_correct_answer is False)]
    export_data = []
    for label in labels:
        answer_text = label.answer.answer if label and label.answer else ""
        offset_start_in_document = 0
        if label.answer and label.answer.offsets_in_document:
            offset_start_in_document = label.answer.offsets_in_document[0].start
        if full_document_context:
            context = label.document.content
            answer_start = offset_start_in_document
        else:
            text = label.document.content
            # the final length of context(including the answer string) is 'context_size'.
            # we try to add equal characters for context before and after the answer string.
            # if either beginning or end of text is reached, we correspondingly
            # append more context characters at the other end of answer string.
            context_to_add = int((context_size - len(answer_text)) / 2)
            start_pos = max(offset_start_in_document - context_to_add, 0)
            additional_context_at_end = max(context_to_add - offset_start_in_document, 0)
            end_pos = min(offset_start_in_document + len(answer_text) + context_to_add, len(text) - 1)
            additional_context_at_start = max(offset_start_in_document + len(answer_text) + context_to_add - len(text), 0)
            start_pos = max(0, start_pos - additional_context_at_start)
            end_pos = min(len(text) - 1, end_pos + additional_context_at_end)
            context = text[start_pos:end_pos]
            answer_start = offset_start_in_document - start_pos
        squad_label: Dict[str, Any]
        if label.is_correct_answer is False and label.is_correct_document is False:  # No answer
            squad_label = {"paragraphs": [{
                        "context": context,
                        "id": label.document.id,
                        "qas": [{"question": label.query, "id": label.id, "is_impossible": True, "answers": []}],}]}
        else:
            squad_label = {"paragraphs": [{
                        "context": context,
                        "id": label.document.id,
                        "qas": [{
                                "question": label.query,
                                "id": label.id,
                                "is_impossible": False,
                                "answers": [{"text": answer_text, "answer_start": answer_start}],}],}]}
            # quality check
            start = squad_label["paragraphs"][0]["qas"][0]["answers"][0]["answer_start"]
            answer = squad_label["paragraphs"][0]["qas"][0]["answers"][0]["text"]
            context = squad_label["paragraphs"][0]["context"]
            if not context[start : start + len(answer)] == answer:
                print(f"Skipping invalid squad label as string via offsets ",f"('{context[start:start + len(answer)]}') does not match answer string ('{answer}') ")
        export_data.append(squad_label)
    export = {"data": export_data}
    with open("feedback_squad_direct.json", "w", encoding="utf8") as f:
        json.dump(export_data, f, ensure_ascii=False, sort_keys=True, indent=4)
    return export

In [62]:
response = client.post("/feedback/",
    json={
      'query': 'How many sessions were planned at the Fourth European Conference on Space Debris?',
      'document': {
        'content': 'The 5th IAASS Conference was held  17 19 October in Versailles, France.  The 3 day  conference consisted of  forty 2 hour sessions  devoted to a number of  general space safety  issues including three sessions on spacecraft  reentry, four sessions on space traffic, two  sessions on space debris, one session on debris ',
        'content_type': 'text',
        'id': '9c91054b96e9267e1f3476ea6467980a',
        'meta': {
          'article_title': 'nasa_odqn_paper_295_m.txt'
        }
      },
      'is_correct_answer': True,
      'is_correct_document': True,
      'origin': 'user-feedback',
      'answer': {
        'answer': 'two',
        'type': 'extractive',
        'score': 0.4920458346605301,
        'context': ' safety  issues including three sessions on spacecraft  reentry, four sessions on space traffic, two  sessions on space debris, one session on debris ',
        'offsets_in_document': [{
          'start': 266,
          'end': 269
        }],
        'offsets_in_context': [{
          'start': 97,
          'end': 100
        }],
        'document_id': '9c91054b96e9267e1f3476ea6467980a',
        'meta': {
          'article_title': 'nasa_odqn_paper_295_m.txt'
        }
      }
    },
)
response.status_code, response.json()


(200, None)

In [ ]:
response = client.get("/feedback/")
print(response.status_code)
response.json()

In [ ]:
response = client.delete("/feedback/")
print(response.status_code)
response.json()

In [64]:
response = client.post("/eval-feedback/")
print(response.status_code)
response.json()

200


{'answer_accuracy': 1.0, 'document_accuracy': 1.0, 'n_feedback': 2}

In [65]:
response = client.get("/export-feedback/")
print(response.status_code)
response.json()

200


{'data': [{'paragraphs': [{'context': 'The 5th IAASS Conference was held  17 19 October in Versailles, France.  The 3 day  conference consisted of  forty 2 hour sessions  devoted to a number of  general space safety  issues including three sessions on spacecraft  reentry, four sessions on space traffic, two  sessions on space debris, one session on debris ',
     'id': '9c91054b96e9267e1f3476ea6467980a',
     'qas': [{'answers': [{'answer_start': 266, 'text': 'two'}],
       'id': 'e194b0d2-4e98-499a-90fe-da0fccbf1c39',
       'is_impossible': False,
       'question': 'How many sessions were planned at the Fourth European Conference on Space Debris?'}]}]},
  {'paragraphs': [{'context': 'The 5th IAASS Conference was held  17 19 October in Versailles, France.  The 3 day  conference consisted of  forty 2 hour sessions  devoted to a number of  general space safety  issues including three sessions on spacecraft  reentry, four sessions on space traffic, two  sessions on space debris, one se

In [44]:
from pyngrok import ngrok
ngrok.set_auth_token("2Cic2NFLJHneI0vmcTJfWs46ZQU_53MiGMPf8CJkdtvYw5JAJ")

In [45]:
from colabcode import ColabCode
server = ColabCode(port=10000, code=False)

In [ ]:
'https://31a3-35-229-214-25.ngrok.io'

In [67]:
server.run_app(app=app)

Public URL: NgrokTunnel: "https://31a3-35-229-214-25.ngrok.io" -> "http://localhost:10000"


INFO:     Started server process [61]
INFO - uvicorn.error -  Started server process [61]
INFO:     Waiting for application startup.
INFO - uvicorn.error -  Waiting for application startup.
INFO:     Application startup complete.
INFO - uvicorn.error -  Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:10000 (Press CTRL+C to quit)
INFO - uvicorn.error -  Uvicorn running on http://127.0.0.1:10000 (Press CTRL+C to quit)


INFO:     54.155.47.149:0 - "GET /initialized HTTP/1.1" 201 Created
INFO:     54.155.47.149:0 - "GET /initialized HTTP/1.1" 201 Created
INFO:     54.155.47.149:0 - "GET /initialized HTTP/1.1" 201 Created
INFO:     54.155.47.149:0 - "GET /initialized HTTP/1.1" 201 Created
INFO:     54.155.47.149:0 - "GET /initialized HTTP/1.1" 201 Created
INFO:     54.155.47.149:0 - "POST /query HTTP/1.1" 307 Temporary Redirect
--> Running a query: query='How many sessions were planned at the Fourth European Conference on Space Debris?' params={'filters': {}, 'Retriever': {'top_k': 3}, 'Reader': {'top_k': 3}} debug=False
INFO:     54.155.47.149:0 - "GET /initialized HTTP/1.1" 201 Created
INFO:     54.155.47.149:0 - "POST /query HTTP/1.1" 307 Temporary Redirect
--> Running a query: query='How many sessions were planned at the Fourth European Conference on Space Debris?' params={'filters': {}, 'Retriever': {'top_k': 3}, 'Reader': {'top_k': 3}} debug=False
--> Result from query: {'query': 'How many session

INFO:     Shutting down
INFO - uvicorn.error -  Shutting down
INFO:     Finished server process [61]
INFO - uvicorn.error -  Finished server process [61]
